In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import os.path as op
import time

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback
from tensorflow import set_random_seed
from tqdm import tqdm_notebook

from data import keras_im_generator, im_generator_BSD68
from evaluate import psnr, ssim
from learned_wavelet import learned_wavelet

Using TensorFlow backend.


In [4]:
set_random_seed(1)

In [5]:
batch_size = 1
noise_std = 30 / 255
source = 'bsd68'
validation_split = 0.1
if 'cifar' in source:
    n_samples_train = 5*1e4
    size = 32
elif 'mnist' in source:
    n_samples_train = 6*1e4
    size = 28
elif source == 'bsd68':
    bsd_dir_train = 'BSDS300/images/train'
    n_samples_train = 200
    size = None
if source in ['cifar', 'cifar_grey', 'mnist']:
    im_gen_train = keras_im_generator(mode='training', validation_split=0.1, batch_size=batch_size, source=source, noise_std=noise_std)
    im_gen_val = keras_im_generator(mode='validation', validation_split=0.1, batch_size=batch_size, source=source, noise_std=noise_std)
elif source == 'bsd68':
    im_gen_train = im_generator_BSD68(path=bsd_dir_train, mode='training', validation_split=0.1, batch_size=batch_size, noise_std=noise_std)
    im_gen_val = im_generator_BSD68(path=bsd_dir_train, mode='validation', validation_split=0.1, batch_size=batch_size, noise_std=noise_std)

In [6]:
run_params = {
    'n_scales': 4, 
    'n_details': 3, 
    'n_coarse': 1, 
    'n_groupping': 3,
}
n_epochs = 50
run_id = f'unet_{source}_{int(noise_std * 255)}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

unet_bsd68_30_1570188329


In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=1000, save_weights_only=False)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [8]:
model = learned_wavelet(input_size=(size, size, 3), lr=1e-3, **run_params)
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, None, None, 1 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, None, None, 1 0           conv2d_2[0][0]                   
_____________________________________

In [9]:
# model.fit_generator(
#     im_gen_train, 
#     steps_per_epoch=int((1-validation_split) * n_samples_train / batch_size), 
#     epochs=n_epochs,
#     validation_data=im_gen_val,
#     validation_steps=int(validation_split * n_samples_train / batch_size),
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback,],
# #     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
# )

In [ ]:
%%time
# overfitting trials
data = next(im_gen_train)
val_data = next(im_gen_val)
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0].shape[0], 
    epochs=500, 
    verbose=2, 
    shuffle=False,
)
print('Original metrics')
print(psnr(*data))

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
 - 7s - loss: 14.9278 - keras_psnr: 7.7098 - keras_ssim: -2.9352e-02
Epoch 2/500
 - 1s - loss: 5.8322 - keras_psnr: 11.6749 - keras_ssim: -2.7934e-03
Epoch 3/500
 - 1s - loss: 2.8097 - keras_psnr: 14.7204 - keras_ssim: 0.0053
Epoch 4/500
 - 1s - loss: 1.6871 - keras_psnr: 16.8080 - keras_ssim: 0.0112
Epoch 5/500
 - 1s - loss: 1.2218 - keras_psnr: 18.1783 - keras_ssim: 0.0219
Epoch 6/500
 - 1s - loss: 1.1254 - keras_psnr: 18.5354 - keras_ssim: 0.0301
Epoch 7/500
 - 1s - loss: 1.0887 - keras_psnr: 18.6917 - keras_ssim: 0.0458
Epoch 8/500
 - 1s - loss: 1.0577 - keras_psnr: 18.8227 - keras_ssim: 0.0609
Epoch 9/500
 - 1s - loss: 1.0258 - keras_psnr: 18.9532 - keras_ssim: 0.0733
Epoch 10/500
 - 1s - loss: 1.0056 - keras_psnr: 19.0329 - keras_ssim: 0.0801
Epoch 11/500
 - 1s - loss: 0.9958 - keras_psnr: 19.0688 - keras_ssim: 0.0821
Epoch 12/500
 - 1s - loss: 0.9878 - keras_psnr: 19.0987 - keras_ssim: 0.0821
Epoch 13/500
 - 1s - loss: 

Epoch 104/500
 - 1s - loss: 0.8293 - keras_psnr: 20.0439 - keras_ssim: 0.1839
Epoch 105/500
 - 1s - loss: 0.8289 - keras_psnr: 20.0428 - keras_ssim: 0.1828
Epoch 106/500
 - 1s - loss: 0.8285 - keras_psnr: 20.0459 - keras_ssim: 0.1824
Epoch 107/500
 - 1s - loss: 0.8269 - keras_psnr: 20.0572 - keras_ssim: 0.1853
Epoch 108/500
 - 1s - loss: 0.8253 - keras_psnr: 20.0698 - keras_ssim: 0.1868
Epoch 109/500
 - 1s - loss: 0.8251 - keras_psnr: 20.0684 - keras_ssim: 0.1845
Epoch 110/500
 - 1s - loss: 0.8248 - keras_psnr: 20.0662 - keras_ssim: 0.1850
Epoch 111/500
 - 1s - loss: 0.8235 - keras_psnr: 20.0783 - keras_ssim: 0.1868
Epoch 112/500
 - 1s - loss: 0.8220 - keras_psnr: 20.0886 - keras_ssim: 0.1878
Epoch 113/500
 - 1s - loss: 0.8214 - keras_psnr: 20.0934 - keras_ssim: 0.1890
Epoch 114/500
 - 1s - loss: 0.8211 - keras_psnr: 20.0963 - keras_ssim: 0.1883
Epoch 115/500
 - 1s - loss: 0.8204 - keras_psnr: 20.0946 - keras_ssim: 0.1881
Epoch 116/500
 - 1s - loss: 0.8188 - keras_psnr: 20.1093 - keras

 - 1s - loss: 0.7743 - keras_psnr: 20.3885 - keras_ssim: 0.2150
Epoch 210/500
 - 1s - loss: 0.7744 - keras_psnr: 20.3984 - keras_ssim: 0.2171
Epoch 211/500
 - 1s - loss: 0.7739 - keras_psnr: 20.3862 - keras_ssim: 0.2162
Epoch 212/500
 - 1s - loss: 0.7739 - keras_psnr: 20.3689 - keras_ssim: 0.2129
Epoch 213/500
 - 1s - loss: 0.7721 - keras_psnr: 20.4063 - keras_ssim: 0.2164
Epoch 214/500
 - 1s - loss: 0.7728 - keras_psnr: 20.4148 - keras_ssim: 0.2192
Epoch 215/500
 - 1s - loss: 0.7714 - keras_psnr: 20.4008 - keras_ssim: 0.2168
Epoch 216/500
 - 1s - loss: 0.7718 - keras_psnr: 20.3861 - keras_ssim: 0.2137
Epoch 217/500
 - 1s - loss: 0.7707 - keras_psnr: 20.4227 - keras_ssim: 0.2200
Epoch 218/500
 - 1s - loss: 0.7705 - keras_psnr: 20.4265 - keras_ssim: 0.2201
Epoch 219/500
 - 1s - loss: 0.7702 - keras_psnr: 20.4015 - keras_ssim: 0.2142
Epoch 220/500
 - 1s - loss: 0.7691 - keras_psnr: 20.4128 - keras_ssim: 0.2169
Epoch 221/500
 - 1s - loss: 0.7692 - keras_psnr: 20.4363 - keras_ssim: 0.2212


In [ ]:
data[0].shape